In [ ]:
# Translated PINN class from TensorFlow 1.x to PyTorch

import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import grad
import numpy as np
import os
from typing import List, Dict

import torch
import torch.nn as nn
import numpy as np
from typing import List

from utils.constants import * 
from config.settings import * 
from scipy.optimize import minimize
import os


class PINN(nn.Module):
    def __init__(self, x: np.ndarray, y: np.ndarray, t: np.ndarray,
                 v1: np.ndarray, v5: np.ndarray, layers: List[int],
                 use_pde: bool = True):
        super(PINN, self).__init__()

        # Data
        self.x = torch.tensor(x, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)
        self.t = torch.tensor(t, dtype=torch.float32)
        self.v1 = torch.tensor(v1, dtype=torch.float32)
        self.v5 = torch.tensor(v5, dtype=torch.float32)
        self.layers = layers
        self.use_pde = use_pde

        # Normalization constants (assume you define DIFF_NORMS externally)
        self.diff_norms = DIFF_NORMS

        # Bounds
        X = torch.cat([self.x, self.y, self.t], dim=1)
        self.lb = X.min(dim=0).values
        self.ub = X.max(dim=0).values

        # Define neural networks
        self.net_v1 = self.build_network(layers)
        self.net_v5 = self.build_network(layers)

        if use_pde:
            self.net_v2 = self.build_network(layers)
            self.net_v3 = self.build_network(layers)
            self.net_v4 = self.build_network(layers)
            self.loss_history = {'v1': [], 'v5': [], 'f1': [], 'f5': []}
        else:
            self.loss_history = {'v1': [], 'v5': []}

        # Optimizer placeholders (setup outside based on parameters)
        self.optimizers = {}

    def build_network(self, layers: List[int]):
        """Build a fully connected neural network with Tanh activation."""
        net = []
        for i in range(len(layers) - 1):
            net.append(nn.Linear(layers[i], layers[i + 1]))
            if i != len(layers) - 2:
                net.append(nn.Tanh())
        return nn.Sequential(*net)
        
    def setup_device(self):
        """Setup PyTorch device configuration."""
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
        # Move models to device
        self.net_v1.to(self.device)
        self.net_v5.to(self.device)
        
        if self.use_pde:
            self.net_v2.to(self.device)
            self.net_v3.to(self.device)
            self.net_v4.to(self.device)
    
        # Also move data tensors to device
        self.x = self.x.to(self.device)
        self.y = self.y.to(self.device)
        self.t = self.t.to(self.device)
        self.v1 = self.v1.to(self.device)
        self.v5 = self.v5.to(self.device)

    